In [56]:
import requests
import os
import pandas as pd
import spacy
import plotly.express as px
import numpy as np
import pytextrank
from bs4 import BeautifulSoup

In [6]:
prob_df = pd.read_csv(r"C:\Users\Ravit\Documents\rnd\horizon_scanning_lab\articles\analyze_Articles\reddit_articles_analysis\reddit_problematic_articles.txt", sep=" ")

In [8]:
prob_df = prob_df.T

In [16]:
prob_urls = prob_df.index.to_list()

In [47]:
for i, url in enumerate(prob_urls):
    prob_urls[i] = url.split('"')[0].strip(',').strip('[').strip(']').replace("'","")

In [51]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

In [66]:
#get websites. This is to learn what parsing tags are required to extract text

websites = []
for url in prob_urls:
    websites.append(url.split('/')[2])

In [238]:
websites = list(set(websites))

In [239]:
from collections import Counter

Counter(websites)

Counter({'siliconangle.com': 1,
         'www.thegamer.com': 1,
         'www.science.org': 1,
         'www.wsj.com': 1,
         'www.tandfonline.com': 1,
         'www.abc.net.au': 1,
         'apnews.com': 1,
         'onlinelibrary.wiley.com': 1,
         'www.sciencedirect.com': 1,
         'www.autoevolution.com': 1,
         'www.pnas.org': 1,
         'www.techdirt.com': 1,
         'www.newsweek.com': 1,
         'agupubs.onlinelibrary.wiley.com': 1,
         'www.tomshardware.com': 1,
         'thehill.com': 1,
         'doi.org': 1,
         'journals.sagepub.com': 1,
         'www.uq.edu.au': 1,
         'www.fiercebiotech.com': 1,
         'www.lightreading.com': 1,
         'academic.oup.com': 1})

Find parsers


'siliconangle.com'
title = soup.find('h3', {'class': 'sa-post-title'}).text
body = soup.find('div', {'class': 'single-post-content'}).find_all('p')
text = "".join([p.find_next(text=True).strip() for p in body])


'www.newsweek.com' 
title =  soup.find('h1', {'class':'title'}).text
body = soup.find('div', {'class':'article-body v_text'}).find_all('p')
text = "".join([p.find_next(text=True).strip() for p in body])

'www.thegamer.com'
title = soup.find('h1', {'class':'heading_title'}).text
body = soup.find('section', {'id':'article-body'}).find_all('p')
text = "".join([p.find_next(text=True).strip() for p in body])

'apnews.com'
title = soup.find('h1', {"class":"Component-heading-0-2-16"}).text
body = soup.find('div', {"class":"Article"}).find_all('p')
text = "".join([p.find_next(text=True).strip() for p in body])

'www.techdirt.com'
#didn't succeed

'agupubs.onlinelibrary.wiley.com'
#didn't succeed

'www.uq.edu.au'
title = soup.find('h1', {'id':"page-title"}).text
body = soup.find('div', {'id':"content"}).find_all('p')
text = "".join([p.find_next(text=True).strip() for p in body])

'www.fiercebiotech.com'
#didn't succeed

academic.oup.com
#didn't succeed

www.science.org
#didn't succeed

www.wsj.com
title = soup.find('h1', {"class":"css-1lvqw7f-StyledHeadline e1ipbpvp0"}).text
text = soup.find('section', {'class':'css-az2xkl-Container-Container e1d75se20'}).text

ww.tandfonline.com
title = soup.find("span", {"class":"NLM_article-title hlFld-title"}).text
text = soup.find("div", {"class":"abstractSection abstractInFull"}).text

www.abc.net.au
title = soup.find("h1", {"class":"_1EAJU hMmqO WL4Yr n-Wqw _18EFj _2ZOIT _3HiTE x9R1x pDrMR hmFfs _390V1"}).text
body = soup.find("div", {"class":"_3P3cP _3sFAh"}).find_all('p')
text = "".join([p.find_next(text=True).strip() for p in body])

onlinelibrary.wiley.com
#didn't succeed

'www.sciencedirect.com'
#didn't succeed

www.autoevolution.com
#didn't succeed

www.pnas.org
#didn't succeed

www.tomshardware.com
title=soup.find("title").text
body = soup.find("div",{"id":"article-body"})
text = "".join([p.find_next(text=True).strip() for p in body])

thehill.com
title=soup.find("title").text
body = soup.find("div", {"class":"article__text | body-copy | flow"}).find_all('p')
text = "".join([p.find_next(text=True).strip() for p in body])



journals.sagepub.com
#not working

www.lightreading.com
#not working


In [245]:
#websites that work with requests
request_webs = ['siliconangle.com', 'www.newsweek.com', 'www.thegamer.com',  'apnews.com', 'www.uq.edu.au', 'www.wsj.com', 
'www.tandfonline.com', 'www.abc.net.au', 'www.tomshardware.com', 'thehill.com']

#websites that don't
no_requests_webs = ['www.techdirt.com', 'agupubs.onlinelibrary.wiley.com', 'www.fiercebiotech.com', 'academic.oup.com', 'www.science.org', 'onlinelibrary.wiley.com', 'www.sciencedirect.com', 'www.autoevolution.com', 'www.pnas.org', 'journals.sagepub.com', 'www.lightreading.com']

#doi.org => link not extracted properly
#remove from list
prob_urls.remove('https://doi.org/10.1016/j.humov.2022.103016')

In [ ]:

for url in prob_urls:
    web = url.split('/')[2] 
    if web in request_webs:
        result = requests.get(url)
        soup = BeautifulSoup(result.text)

        if web == 'siliconangle.com':
            title = soup.find('h3', {'class': 'sa-post-title'}).text
            body = soup.find('div', {'class': 'single-post-content'}).find_all('p')
            text = "".join([p.find_next(text=True).strip() for p in body])

        if web=='www.newsweek.com':
            title =  soup.find('h1', {'class':'title'}).text
            body = soup.find('div', {'class':'article-body v_text'}).find_all('p')
            text = "".join([p.find_next(text=True).strip() for p in body])
            
        if web=='www.thegamer.com':
            title = soup.find('h1', {'class':'heading_title'}).text
            body = soup.find('section', {'id':'article-body'}).find_all('p')
            text = "".join([p.find_next(text=True).strip() for p in body])
            
        if web=='apnews.com':
            title = soup.find('h1', {"class":"Component-heading-0-2-16"}).text
            body = soup.find('div', {"class":"Article"}).find_all('p')
            text = "".join([p.find_next(text=True).strip() for p in body])
            
        if web=='www.uq.edu.au':
            title = soup.find('h1', {'id':"page-title"}).text
            body = soup.find('div', {'id':"content"}).find_all('p')
            text = "".join([p.find_next(text=True).strip() for p in body])
            
        if web=='www.wsj.com':
            title = soup.find('h1', {"class":"css-1lvqw7f-StyledHeadline e1ipbpvp0"}).text
            text = soup.find('section', {'class':'css-az2xkl-Container-Container e1d75se20'}).text
            
        if web=='www.tandfonline.com':
            title = soup.find("span", {"class":"NLM_article-title hlFld-title"}).text
            text = soup.find("div", {"class":"abstractSection abstractInFull"}).text
            
        if web=='www.abc.net.au':
            title = soup.find("h1", {"class":"_1EAJU hMmqO WL4Yr n-Wqw _18EFj _2ZOIT _3HiTE x9R1x pDrMR hmFfs _390V1"}).text
            body = soup.find("div", {"class":"_3P3cP _3sFAh"}).find_all('p')
            text = "".join([p.find_next(text=True).strip() for p in body])
            
        if web=='www.tomshardware.com':
            title=soup.find("title").text
            body = soup.find("div",{"id":"article-body"})
            text = "".join([p.find_next(text=True).strip() for p in body])
        
        if web == 'thehill.com':
            title=soup.find("title").text
            body = soup.find("div", {"class":"article__text | body-copy | flow"}).find_all('p')
            text = "".join([p.find_next(text=True).strip() for p in body])

        


In [235]:
result = requests.get('https://www.lightreading.com/security/the-cloud-and-5g-security-apocalypse-is-only-matter-of-time/d/d-id/781259', headers=headers)

In [236]:
soup = BeautifulSoup(result.text)

In [237]:
soup

<!DOCTYPE html>
<html lang="en-US">
<head>
<title>Just a moment...</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="noindex,nofollow" name="robots"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<link href="/cdn-cgi/styles/challenges.css" rel="stylesheet"/>
</head>
<body class="no-js">
<div class="main-wrapper" role="main">
<div class="main-content">
<h1 class="zone-name-title h1">
<img class="heading-favicon" onerror="this.onerror=null;this.parentNode.removeChild(this)" src="/favicon.ico"/>
            www.lightreading.com
        </h1>
<h2 class="h2" id="challenge-running">
            Checking if the site connection is secure
        </h2>
<noscript>
<div id="challenge-error-title">
<div class="h2">
<span class="icon-wrapper">
<div class="heading-icon warning-icon"></div>
</span>
<span id="challenge-error-text">
                        Enable JavaScript and cooki

In [230]:
soup.find("div", {"class":"article__text | body-copy | flow"}).find_all('p')

[<p>NASA has suggested an experimental cooling system it is funding could ultimately allow electric vehicle users to charge their cars within five minutes.</p>,
 <p>The agency said a team led by a Purdue University professor has developed the “subcooled flow boiling” technology for experimentation, with the hope it can control future systems’ temperatures in space.</p>,
 <p>“A team sponsored by NASA’s Biological and Physical Sciences Division is developing a new technology that will not only achieve orders-of-magnitude improvement in heat transfer to enable these systems to maintain proper temperatures in space, but will also enable significant reductions in size and weight of the hardware,” NASA <a href="https://science.nasa.gov/technology/technology-highlights/cooling-technique-developed-for-space-use-makes-charging-electric-cars-on-earth-quicker-and-easier">said in a blog post last week</a>. </p>,
 <p>“What’s more, this same technology may make owning an electric-powered car here on

In [231]:
body = soup.find("div", {"class":"article__text | body-copy | flow"}).find_all('p')
text = "".join([p.find_next(text=True).strip() for p in body])
text

'NASA has suggested an experimental cooling system it is funding could ultimately allow electric vehicle users to charge their cars within five minutes.The agency said a team led by a Purdue University professor has developed the “subcooled flow boiling” technology for experimentation, with the hope it can control future systems’ temperatures in space.“A team sponsored by NASA’s Biological and Physical Sciences Division is developing a new technology that will not only achieve orders-of-magnitude improvement in heat transfer to enable these systems to maintain proper temperatures in space, but will also enable significant reductions in size and weight of the hardware,” NASA“What’s more, this same technology may make owning an electric-powered car here on Earth easier and more feasible,” the post continued.NASA indicated achieving such a feat of charging electric vehicles within five minutes would require chargers to provide current at 1,400 amperes, far higher than currently available 

In [166]:
"".join([p.find_next(text=True).strip() for p in body])

'Research from The University of Queensland has found no link between community water fluoridation and adverse effects on children’s brain development.Professor Loc Do“We found emotional and behavioural development, and functions such as memory and self-control, were at least equivalent to those who had no exposure to fluoridated water,” Professor Do said.“In other words, there was no difference in child development and function related to fluoridated water.“This finding shows that consuming water with fluoride at levels used for public supplies in Australia is safe and it supports continuing and expanding fluoridation programs.”Currently, approximately 90 per cent of the Australian population has access to fluoridated water, although in Queensland it is 71 per cent.Many regional Queensland areas and Aboriginal and Torres Strait Islander communities are not covered by a fluoridation program.“A small but vocal group of people sometimes claims that water fluoridation can have adverse neu

In [146]:
soup.find('h1', {"class":"posttitle"})

In [139]:
soup.find('div', {"class":"Article"}).find_all('p')

[<p class="Component-root-0-2-46 p Component-p-0-2-37">WASHINGTON (AP) — Nearly 400 school districts spanning all 50 states and Washington, D.C., along with several tribes and U.S. territories, are receiving roughly $1 billion in grants to purchase about 2,500 “clean” school buses under a new federal program.</p>,
 <p class="Component-root-0-2-46 p Component-p-0-2-37">The Biden administration is making the grants available as part of a wider effort to accelerate <a class="paragraph-link" href="/article/technology-california-clean-air-act-vehicle-emissions-standards-eebb48c13e24835f2c5b9cb56796182a">the transition to zero-emission vehicles</a><!-- --> and reduce air pollution near schools and communities. </p>,
 <p class="Component-root-0-2-46 p Component-p-0-2-37">Vice President <a class="paragraph-link" href="/hub/kamala-harris">Kamala Harris</a><!-- --> and <a class="paragraph-link" href="/hub/us-environmental-protection-agency">Environmental Protection Agency</a><!-- --> Administrat

In [70]:
prob_urls

['https://www.newsweek.com/baby-alpaca-learns-walk-again-prosthetic-leg-adorable-clip-1749410?',
 'https://academic.oup.com/humupd/article/28/4/457/6555833?login=false',
 'https://www.fiercebiotech.com/research/ut-austin-scientists-design-safer-cas9-improved-crispr-gene-editing-accuracy',
 'https://www.wsj.com/articles/metas-facebook-says-its-new-vr-headset-could-replace-workers-pcs-11665521118?st=w9xk0vs5je9lyo1&reflink=share_mobilewebshare',
 'https://www.sciencedirect.com/science/article/pii/S1877050922012777',
 'https://www.lightreading.com/security/the-cloud-and-5g-security-apocalypse-is-only-matter-of-time/d/d-id/781259',
 'https://www.abc.net.au/news/science/2022-10-26/extreme-miyake-radiation-events-tree-rings-solar-storms/101563738',
 'https://onlinelibrary.wiley.com/doi/full/10.1002/smj.3459',
 'https://www.wsj.com/articles/solar-rollout-rouses-resistance-in-europes-countryside-11665234001',
 'https://www.autoevolution.com/news/harvard-engineers-invent-a-solid-state-battery-t